# Practical Session 3: Running ROS in a Docker container. Hello ROS

## Redes de Sensores y Sistemas Autónomos 
### Grado en Ingeniería de las Tecnologías de Telecomunicación
### Universidad de Sevilla

## Objectives

In this practical session, we will learn the basics of Docker containers and how to use them to run ROS (Robot Operating System). ROS is a set of software libraries and tools that help you build robot applications, including hardware drivers, state-of-the-art algorithms, robotics simulators, powerful developer tools, and much more. ROS is open-source and has a strong support from the robotics community, so you can find a wide spectrum of available packages for your robotics application. Although ROS can run on different platforms, its most widely used and supported platform is Ubuntu. One option would be to install Ubuntu and ROS on your computer, but in this course we will use a more powerful tool, which is Docker containers. In particular, in this session, we'll learn a bit more about:

+ Using Docker for application development.
+ Running ROS on a Docker container.
+ Launch our first simulation in the Gazebo simulator connected to ROS 

## Docker

[Docker](https://www.docker.com/) is an open platform for developing, shipping, and running applications. It enables you to separate your applications from your infrastructure so you can deliver software quickly. Docker allows developers to easily deploy their applications in a _**container**_ running on a host operating system. The container will package and run your application in an isolated environment that can be based on a different operating system. The key benefit of Docker is that it allows users to package an application with all of its dependencies into a standardized unit for software development. Unlike virtual machines, containers do not have high overhead and hence enable more efficient usage of the underlying system and resources. Thus, you can run many containers simultaneously on a given host. As they are lightweight and contain everything needed to run the application, you don't need to rely on what's installed on the host. Moreover, you can share containers while you work, and be sure that everyone you share with gets the same container that works in the same way.


### Main concepts


#### Images

An image is a read-only template with instructions for creating a Docker container. Often, an image is based on another image, with some additional customization. For example, you may build an image which is based on an Ubuntu image, and install ROS and your robotics application on top, as well as the configuration details needed to make your application run. An image is a standardized package that includes all the configuration files, binaries and libraries to run a container. Images are immutable, once created, they cannot be modified. You can only make a new image or add changes on top of it. Moreover, images are composed of layers, and each layer represents a set of file system changes that add, remove, or modify files.


You might use images created by others and published in a registry or create your own images. To build your own image, you create a __Dockerfile__ with a simple syntax for defining the steps needed to create the image and run it. Each instruction in a Dockerfile creates a layer in the image. When you change the Dockerfile and rebuild the image, only those layers which have changed are rebuilt. This is part of what makes images so lightweight, small, and fast, when compared to other virtualization technologies.


#### Docker registries
A Docker registry stores Docker images. [Docker Hub](https://hub.docker.com) is a public registry with over 100,000 images that you can run locally created by developers. By default, Docker searches for images on Docker Hub, but you could also run your own private registry.


#### Containers
A container is a runnable instance of an image. You can create, start, stop, or delete a container using the Docker GUI or CLI. You can run multiple containers from the same image, connect them to the same (or different) network, attach storage to them, etc. By default, a container is relatively well isolated from other containers and its host machine, but this level of isolation can also be configured. 

For further information about how Docker works, check its [documentation](https://docs.docker.com/get-started).


### Installation

__Windows/macOS__

Install Docker Desktop for Windows (Windows 10 or 11 is required) or macOS with the [installer](https://docs.docker.com/get-started/get-docker). Select the WSL backend during installation. 

__Important: Before using Docker commands in the terminal, you should start the Docker Desktop application in the background.__ 

If you find issues when starting Docker Desktop, you may need to enable virtualization features in your computer BIOS, VT-x y VT-d for Intel, AMD-V y AMD-Vi for AMD. Make also sure that Hyper-V is activated. Hyper-V is a Microsoft virtualization tool that comes installed with Windows and is used by WSL. VirtualBox or VMWare have their on hypervisor and may switch off Hyper-V for compatibility. If you find issues running WSL when starting Docker Desktop, make sure that Hyper-V is activated. Run `bcdedit | findstr "hypervisorlaunchtype"` to check whether this feature is off. You can activate it with `bcdedit /set hypervisorlaunchtype auto`.

__Ubuntu__

Install Docker Engine for Ubuntu (Ubuntu 20.04 LTS or newer is required). Follow the instructions [here](https://docs.docker.com/engine/install/ubuntu/), in Section "Install using the apt repository". Then follow [post-install instructions for Linux](https://docs.docker.com/engine/install/linux-postinstall).

### Using Docker containers

In this section, we are going to learn the basic commands to run Docker containers. You can find a cheat sheet with these main commands [here](https://docs.docker.com/get-started/docker_cheatsheet.pdf). First, let's download a simple test Docker image:

In [ ]:
docker pull hello-world 


The `pull` command allows you to pull any image which is present in the official registry of docker, Docker hub. By default, it pulls the latest image, but you can also specify the image version. You can use the command `docker images` to see a list of all images downloaded on your system. Let's now run a Docker container based on this image: 

In [ ]:
docker run hello-world


This creates a new container from the image `hello-world` and starts that container. If it were not present on the system, the image is first pulled. The `docker ps` command shows you all the containers that are currently running; you can use `docker ps -a` to show also not running containers. `hello-world` is a quite simple image that only runs a binary file that prints a message. You should have seen that message after running the container. 

Instead of using the container ID (which looks like an arbitrary number) to refer to a certain container, you can also specify a container name before the image name `docker run --name <container_name> <image_name>`. Let's run a more interesting container:

In [ ]:
docker run --name ubuntu_container -it ubuntu bash

If it is not on your system, the previous command will first pull the `ubuntu` image, which replicates an Ubuntu file system. Using the `run` command with the `-it` flag moves you into an interactive terminal session where you can execute commands inside the container. The `bash` command is specified as the first command for that container, so it will be as if you were in a new bash terminal separate from your host machine and inside the container. If you don't specify `bash` (or another initial command) when running the container, Docker will run the default command defined in the container's image. 

Try to run some Linux commands inside the container to check the Ubuntu file system, e.g., `ls` and `cd`. Then open another terminal in your host machine and list your containers with `docker ps -a`. Note that now your running container is named `ubuntu_container`, which is the name you specified. 

So far, you have a running container with an interactive terminal session to execute commands inside. However, you may need more than one terminal to run multiple commands in parallel in the same container. For that, open another terminal in your host machine and type:

In [ ]:
docker exec -it ubuntu_container bash

The previous command opens a new bash shell inside your running container `ubuntu_container`. At some point, you may want to stop a container because it crashed or to switch to another one. You may also want to start the container again later. To start or stop an existing container, use `docker start|stop <container_name> (or <container_ID>)`.

Docker containers occupy space on your host's disk, so it is good practice to clean up containers once you are done with them. List your containers with `docker ps -a` and use the `docker rm <container_name` command to remove them. You can also delete images that you no longer need by running `docker rmi <image_name>`. For instance, if you had the following containers, once stopped, you could remove them with the command `docker rm ubuntu_container 8987409a69d4`: 

<figure style="text-align:center">
  <img src="images/docker_terminal.png" alt="" width=1100>
  <figcaption>Fig. 1: Example Docker container list.</figcaption>
</figure>

## Running ROS in a Docker container

In this section, we will set up and run our Docker container with ROS. You can build a customized Docker image from a Dockerfile with `docker build -t <image_name>`. A Dockerfile is a simple text file that contains a list of commands that Docker calls while creating an image. It's a simple way to automate the image creation process. The commands you write in a Dockerfile are almost identical to their equivalent Linux commands.

<figure style="text-align:center">
  <img src="images/dockerfile.png" alt="" width=600>
  <figcaption>Fig. 2: Docker working scheme.</figcaption>
</figure>

Have a look at the Dockerfile in the _docker_ folder of the course repository:

In [ ]:
FROM osrf/ros:noetic-desktop-full

SHELL ["/bin/bash", "-c"] 

# Required ROS packages  
RUN sudo apt update && apt install -y ros-noetic-joy ros-noetic-teleop-twist-joy \
    ros-noetic-teleop-twist-keyboard ros-noetic-laser-proc \
    ros-noetic-rgbd-launch ros-noetic-rosserial-arduino \
    ros-noetic-rosserial-python ros-noetic-rosserial-client \
    ros-noetic-rosserial-msgs ros-noetic-amcl ros-noetic-map-server \
    ros-noetic-move-base ros-noetic-urdf ros-noetic-xacro \
    ros-noetic-compressed-image-transport ros-noetic-rqt* ros-noetic-rviz \
    ros-noetic-gmapping ros-noetic-navigation ros-noetic-interactive-markers \
    ros-noetic-vision-msgs

# ROS packages for Turtlebot3 robot     
RUN sudo apt install -y ros-noetic-dynamixel-sdk \
    ros-noetic-turtlebot3-*

# We also install Git and other tools
RUN sudo apt install -y git vim nano iputils-ping net-tools

# MESA drivers for hardware acceleration graphics (Gazebo and RViz)
RUN sudo apt -y install libgl1-mesa-glx libgl1-mesa-dri && \
    rm -rf /var/lib/apt/lists/*

# By default the DockerFile uses the root as user. 
# We will add a custom user so that we work in user space by default
# And then switch to that user
# Source: https://www.docker.com/blog/understanding-the-docker-user-instruction/
RUN useradd -ms /bin/bash rssa
USER rssa

# We source the ROS instalation in user space
RUN echo "source /opt/ros/noetic/setup.bash" >> ~/.bashrc
RUN echo "export TURTLEBOT3_MODEL=burger" >> ~/.bashrc

# We will use the home directory as work directory
# So that the docker starts at HOME!
WORKDIR "/home/rssa"


We will use that Dockerfile to build our image with ROS. Move to the folder containing the Dockerfile and type:

In [ ]:
docker build -t rssa .

This will build a new image called `rssa`. The dot indicates that the Dockerfile is in the current repository. The instructions in the Dockerfile are used to create the image. The following type of commands are included:

+ `FROM`: It specifies a base image to initialize the first layer. We use an official image with a Linux file system and ROS Noetic installed.
+ `SHELL`: It specifies the default shell to run subsequent commands. 
+ `RUN`: It runs a script or command. In particular, our Dockerfile uses it to install a series of additional ROS packages and drivers that will be necessary.
+ `USER`: Changes the user (by default it starts in root user. We  perform installation steps in root and then we switch to a new generated user in the latests steps)
+ `WORKDIR`: It sets the working directory for any following instructions in the Dockerfile. The container will start in that directory. If the directory specified by WORKDIR doesn't exist, it will be created. 

You can learn all possible instructions in Dockerfiles [here](https://docs.docker.com/reference/dockerfile/). Other common instructions not used in our Dockerfile are the following:

+ `COPY`: It copies new files or directories and adds them to the file system of the image.
+ `CMD`: It tells the container which command to run when started. This command can be overriden when launching a container.
+ `ENV`: It sets an environment variable. This variable will be accessible during the build process and once the container is run.

### Running the container with a bind mount and GUI support

Running our container and executing a terminal sounds cool, but there are a couple of additional features that would be quite helpful: 1) how to exchange data with the container; and 2) how to run graphical applications in the container. This would allow us to load our code into the container and then retreive any data generated within it. Additionally, we could run applications that use graphics, such as simulators or visualization tools. Fortunately, Docker provides solutions for both issues. 

Docker allows you to create a virtual volume for a container, where your data can persist after stopping the container. A volume mount is a great choice when you need somewhere persistent to store your application data. A __bind mount__ is another type of mount, which allows you to share a directory from the host's file system with the container. For instance, you can use this shared directory to store your workspace with your code. The container will see the changes you make to the code immediately, as soon as you save a file, but you won't need to copy that code into the container. You can check more information about bind mounts in Docker [here](https://docs.docker.com/get-started/workshop/06_bind_mounts/). 

In short, you can create a link between a directory in the host machine and a directory in the container by adding the flag `--mount` to the Docker running command:

In [ ]:
docker run -it --mount type=bind,source=$HOME/rssa_shared,target=/home/rssa rssa bash

The previous command creates a link between the directory `rssa_shared` in the host machine and another one in the container, mounted on `/home/rssa`. Therefore, that shared directory will be accessible from both the container and the host.   

__Note 1:__ If you are using Windows, you should change the previous source command to `C:\Users\username\lab_rob_shared`.

__Note 2:__ The idea is to use the container to compile and execute your ROS code, but to edit it on your host computer. There is a Docker extension in VS Code that will allow it to enter the Docker system so that it can search for the ROS installation. 


#### GUI support

Regarding graphical applications, we can enable the use of GUIs within Docker containers by means of the X server. The X server is a windowing system for bitmap displays commonly in Linux operating systems. We can connect a container to the host's X server for display. 

__Note:__ There are other options to run Docker containers with GUI support. In case you need more information about that, check it [here](https://wiki.ros.org/es/docker/Tutorials/GUI). 

__Ubuntu__

If your host machine is running Ubuntu, there is already an X server, so we just need to add some flags when running the Docker container:

In [ ]:
xhost +local:docker
docker run -it \
    --env="DISPLAY=$DISPLAY" \
    --env="QT_X11_NO_MITSHM=1" \
    --volume="/tmp/.X11-unix:/tmp/.X11-unix" \
    --name rssa \
    --net=host \
    --privileged \
    --mount type=bind,source=$HOME/rssa_shared,target=/home/rssa \
    rssa \
    bash

With the previous command, we would run the container connected to the host's X server, to run GUI applications, and with a bind mount to share a directory with the host machine. First, the `xhost` command adjusts the permissions of the X server on your host.
Then we call `docker run` to run a container called `rssa` from our previously created image `rssa`. The first three flags are required to enable graphical applications. The `--mount` flag at the end is to create the shared folder ` lab_rob_shared` between the container and the host.  

Instead of writing or copying the previous command in the terminal each time you want to run the container, we provide a
bash script called `run_container_linux.bash` in the `docker` directory of the course repository. Before running the script, you should edit it to write the correct path of your shared directory in the mount flag and change the script execution permissions:

In [ ]:
chmod a+x run_container_linux.bash
./run_container_linux.bash

__Windows and macOS__

The steps to run GUIs in Docker containers are different if your host machine is Windows or macOS. First, you need to install an X server. On Windows, you can use [Xming](http://www.straightrunning.com/XmingNotes/) (any other X server should also work). If you want to use Xming, go to the web site and download and install in the same directory the __public domain releases of Xming and Xming-fonts__. On macOS, XQuartz is a common choice; you can install it with: `brew install --cask xquartz`. Once you have an X server installed, start it before running your container. For example, Xming can be configured and run with the __XLaunch application on Windows__. Then you can run the Docker container as follows:

In [ ]:
docker run -it ^
--env="DISPLAY=host.docker.internal:0" ^
--name rssa ^
--net=host ^
--privileged ^
--mount type=bind,source=C:\Users\username\rssa_shared,target=/home/rssa ^
lab_rob_image ^
bash

With the previous command, we would run the container connected to the host's X server, to run GUI applications, and with a bind mount to share a directory with the host machine. We call `docker run` to run a container called `rssa` from our previously created image `lab_rob_image`. The first flag is required to enable graphical applications. The `--mount` flag at the end is to create the shared folder `lab_rob_shared` between the container and the host.  

Instead of writing or copying the previous command in the terminal each time you want to run the container, we provide scripts in the `docker` directory of the course repository, `run_container_windows.bat` or `run_container_mac.bash`, depening on your system. Before running the script, you should edit it to write the correct path of your shared directory in the mount flag and change the script execution permissions:

In [ ]:
# For Windows
run_container_windows.bat

# For macOS
chmod a+x run_container_mac.bash
./run_container_mac.bash

__Exercise 1a__

Create in your machine your shared directory called at your home directory `rssa_shared`. Then create the ROS Docker image from the provided Dockerfile and use the aforementioned scripts to run a Docker container with a bind mount and GUI support. 

First you should add some lines to automatically configure ROS at startup in the Docker


_Exercise 1b__


To test the newly generated container, run a ROS simulation of the Turtlebot3 robot with the following commands: 
 

In [ ]:
roslaunch turtlebot3_gazebo turtlebot3_empty_world.launch 

Note that we use the bash terminal for entering commands in the Docker.

The bash terminal automatically executes the '$HOME/.bashrc' script at startup. We will generate one .bashrc so that ROS and the turtlebot simulator are automatically configured at startup.

Yo can do it as follows, or by manually editing (with nano or vim editors) the file .bashrc.

In [ ]:

echo "source /opt/ros/noetic/setup.bash" >> /home/rssa/.bashrc
echo "export TURTLEBOT3_MODEL=burger" >> /home/rssa/.bashrc
source ~/.bashrc


## Starting multiple terminals in a Docker session

In ROS it is quite common to have several terminals running together to bring up several ROS nodes, etc. To start a new terminal when a docker is running, you need to run the following command:

In [ ]:
> docker exec -it rssa bash

__Exercise 1c__


Finally, open another terminal within the same container and run a ROS teleoperation interface to move the robot with the keyboard: 


In [ ]:
roslaunch turtlebot3_teleop turtlebot3_teleop_key.launch

## Summary

In this practical session, you should have learned the following:
+ How to install Docker and how to use it to create images and run containers.
+ How to run our ROS container with GUI support and a shared directory with your code repository.